<img src="./pictures/DroneApp_logo.png" style="float:right; max-width: 180px; display: inline" alt="INSA" />
<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 100px; display: inline" alt="INSA" />

# Equations of sizing scenarios

*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

The purpose of this notebook is to illustrate on an existing drone example the main equations to be taken into account during the sizing. The selected drone is the [MK-Quadro](http://wiki.mikrokopter.de/MK-Quadro) drone of Mikrokopter. The technical informations about the MK-quadro drone can be found in [Appendix 1](A1_QuadroDescription.ipynb) or on the website of [MikroKopter](http://wiki.mikrokopter.de/en/FlightTime) company.

*MK-Quadro from MikroKopter company*  
![Quadro](./pictures/MikroQuadro.jpg)


### Propellers fundamental equations

Propellers can be described in an ideal manner by the [disk momentum theory](https://fenix.tecnico.ulisboa.pt/downloadFile/282093452028191/3-Momentum%20Theory%20in%20hover.pdf). The [dimensional analysis]([https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html#SECTION06374000000000000000]) theory enables to describe with more details the propeller performances thanks to aerodynamic coefficients:
- The thrust: $T = C_{T,h} \rho \Omega^2 A R^2=C_{T,p} \rho n^2 D^4$  which gives $ = k_{TD} \Omega^2$ for a given propeller and a given diameter
- The power: $P = C_{P,h} \rho \Omega^3 A R^3=C_{P,p} \rho n^3 D^5 $ which gives the power $P=k_{PD}\Omega^3$ or the torque $M = k_{PD} \Omega^2$ for a given propeller and a given diameter

**Remark:** Be carefull, litterature shows 2 notations:
* the propeller notation with $n$ the propeller speed in [rev/s], $D$ the diameter in [m], $\rho$  the air density in [$kg/m^3$]
* the helicopter notation with $\Omega$ the propeller speed in [rad/s], $R$ the radius in [m], $A$ the disk area in [$m^2$], $\rho$  the air density in [$kg/m^3$]

> Exercice: compare the characteristics of the following [APC propellers](https://www.apcprop.com/files/PER3_10x45MR.dat) to those of the MikroKopter MK-Quadro ([GWS](https://gwsus.com/english/dealer/dealer_gwsusa.htm)).

Characteristics  | APC Propeller | GWS Propeller
-- | -- | --
Diameter [inch] | 10 | 10 
Pitch [inch] | 4.5 | 4.5 
Thrust coef $C_t$ [-] | 0.1125 | 
Power coef $C_p$ [-]  | 0.0254 | 
Thrust coef $k_{TD}$ [$\frac{N}{(rad/s)^2}$] | | $1.799.10^{-5}$
Torque coefficient $k_{PD}$ [$\frac{N}{(rad/s)^2}$] | | $3.054.10^{-7}$

with: $1$ $inch = 0.0254$ $m$, $\rho=1.18$ $kg/m^3$ 


In [1]:
from math import pi

D_pro = 10. * 0.0254 # [m] Propeller diameter
rho_air = 1.18 # [kg/m^3] Air mass volumic (25°C)

# APC coefficients                                                      
C_t = 0.1125 # [-] Thrust coefficient, propeller notation
C_p = 0.0441 # [-] Power coefficient, propeller notation

K_TD_APC = C_t * rho_air * D_pro**4. / (2*pi)**2. # [N/(rad/s)^2] Thrust coefficient
K_PD_APC = C_p * rho_air * D_pro**5. / (2*pi)**3. # [N/(rad/s)^2] Torque coefficient

print("-- APC")
print("Thrust coefficient K_TD derived from APC Ct coefficient: %.2e N/(rad/s)^2"%K_TD_APC)
print("Thrust coefficient K_PD derived from APC Ct coefficient: %.2e N/(rad/s)^2"%K_PD_APC)

# GWS coefficients 
C_t_GWS = 1.799e-5 / (rho_air * D_pro**4. / (2*pi)**2.) # [-] Thrust coefficient
C_p_GWS = 3.054e-7 / (rho_air * D_pro**5. / (2*pi)**3.) # [-] Thrust coefficient

print("-- GWS")
print("Thrust coefficient C_t derived from GWS K_TD coefficient: %.2e"%C_t_GWS)
print("Thrust coefficient C_p derived from GWS K_PD coefficient: %.2e"%C_p_GWS)

-- APC
Thrust coefficient K_TD derived from APC Ct coefficient: 1.40e-05 N/(rad/s)^2
Thrust coefficient K_PD derived from APC Ct coefficient: 2.22e-07 N/(rad/s)^2
-- GWS
Thrust coefficient C_t derived from GWS K_TD coefficient: 1.45e-01
Thrust coefficient C_p derived from GWS K_PD coefficient: 6.07e-02


### Hover flight scenario

The flight conditions in static hover are no forward speed, no vertical speed. This scenario influences in a very important way the autonomy of the vehicle. The thrust generated by propellers have to compensate the global weight of the drone and the load.

*Hover flight (V=0)*  
![Disk momentum](./pictures/disk_momentum_hover.jpg)

> Questions: with the given technical informations, calculate the requested thrust for each propeller for a no load hover flight.

Characteristics  | MK-MikroKopter drone
-- | -- 
Total mass ready to take off |  1350 g
Number of propellers | 4

In [2]:
from math import sqrt

# Hover scenario
# ------------ 
M_total = 1.35 # [kg] mass of the drone 
N_pro = 4 # [-] number of propellers
D_pro = 2.54e-2*10 # [m] propeller diameter
F_pro = M_total * 9.81 / N_pro # [N] Thrust per propeller

> Questions: with the technical informations of GWS propellers calculate the propeller torque, speed and the mechanical power.


In [3]:
# Propeller characteristics
# -----------------------------

K_w_pro =1.799e-5 # [N/(rad/s)^2] thrust coef
M_w_pro = 3.054e-7 # [N.m/(rad/s)^2] torque coef

# to be compared to the ones calculated with regression
# GWS 10*4.5

D_pro = 10 * .0254 #[m] diameter

# Torque and RPM of motors
# -----------------------

omega_pro = sqrt(F_pro/K_w_pro) # [rad/s] Propeller Speed

T_pro = M_w_pro * omega_pro**2.  # [N.m] Propeller torque

print("----------------")
print("Propeller speed: %.2f rad/s or %.0f RPM"%(omega_pro, omega_pro/2./3.14*60.))
print("Propeller torque: %.2f N.m "%(T_pro))
print("Propeller power: %.2f W "%(T_pro * omega_pro))

----------------
Propeller speed: 429.00 rad/s or 4099 RPM
Propeller torque: 0.06 N.m 
Propeller power: 24.11 W 


> Questions: with the following technical informations calculate the motor voltage and current, the autonomy of the drone if we assume an efficiency of 95% for electronic speed controllers (ESC) and a depth of discharge of 80% for the battery.

Component |Characteristic | Value
--- | --- | --- 
Motor | Poles pair number| 7  
| Load current | 6‐9A (DC)  
| Max current | 10A (DC)  
| Speed constant Kv | 760 tr/min/V  
| Mechanical power | 110 W  
| Dimensions | 28.8 x 29 mm  
| Max efficiency | 76 %  
| mass | 65 g  
| Resistance | R = 0.26 ohm  
| Inertia | J= 2.5e-5 kg.m²  
Battery| Voltage | LIPO 4S (4*3.7V)
| Capacity | 3300mAh
| mass | 329g


In [9]:
# Voltage / current calculation
# -----------------------------
# Torque coeff and electrical resistance
K_mot=(1./760.) * 60./2./pi # [V/(rad/s)] Motor torque coeff
R_mot = 0.26 # [Ohm] Motor resistance

# Voltage / current calculation
I_mot = T_pro / K_mot # [A] Current of the motor
U_mot = R_mot * I_mot + omega_pro * K_mot # [V] Voltage of the motor

print("----------------")
print("Motor current: %.2f A "% I_mot)
print("Motor voltage: %.2f V "% U_mot)

# Autonomy of the battery
U_bat = 4.*3.7 # [V] Voltage of the battery
C_bat = 3.3 # [A.h] Capacity  of the battery


nu_esc = 0.95 # [-] Efficiency of the ESC
I_bat = (U_mot * I_mot * N_pro) / nu_esc / U_bat # Battery de-rating current

soc_min = 0.2 # Minimum state of charge (SOC) to respect
t_hov = (1. - soc_min) * C_bat / I_bat # [h] Autonomy of the battery

print("----------------")
print("Battery voltage: %.2f V " % U_bat)
print("Autonomy of the battery : %.1f min" % (t_hov * 60.))

----------------
Motor current: 4.47 A 
Motor voltage: 6.55 V 
----------------
Battery voltage: 14.80 V 
Autonomy of the battery : 19.0 min


### Take off flight scenario
The take off flight scenario is a high power mission segment. This scenario is considered as part of the static scenarios and provides the vehicle with the initial thrust to lift the vehicle. No vertical speed is considered so far.

In this sense, the maximum thrust will be defined as $F_{to}=F_{hover}\cdot k_{max,thrust}$, where $k_{max,thrust}$ is the ratio between max thrust and hover thrust, which is always greater than one.

Propeller diameter or frame stress will be designed to withstand some transitory extreme criteria, such as the maximum thrust.

### Vertical climb scenario
For the vertical climb scenario we consider the effect of the air climb speed for the drone design. 

For this mission, the climb thrust will be defined as: $F_{cl}=\frac{1}{2}\cdot C_X\cdot A_{top} \cdot \rho \cdot V_{cl}^2$

with $C_X$ as drag coefficient, $A_{top} $ as top surface (these both parameters defined by the drone geometry), $\rho$ as the air density and $V_{cl}$, as the climb speed.

For the vertical climb, advance ratio ($J=\frac{V_{cl}}{n_{cl}   D_{pro}}$) is a key parameter to estimate the vertical speed. 
